In [ ]:
!git clone https://github.com/RightNow-AI/rightnow-arabic-llm-corpus.git

Cloning into 'rightnow-arabic-llm-corpus'...
remote: Enumerating objects: 11922, done.
remote: Counting objects: 100% (11922/11922), done.
remote: Compressing objects: 100% (11914/11914), done.
remote: Total 11922 (delta 14), reused 11916 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (11922/11922), 14.54 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (11889/11889), done.
Filtering content: 100% (11883/11883), 2.75 GiB | 5.18 MiB/s, done.


In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import glob
from tqdm import tqdm

# ----------------------------
# Load your 5000 documents
# ----------------------------
paths = sorted(glob.glob("rightnow-arabic-llm-corpus/dataset/arabic_text_*.jsonl"))
docs = []

for p in paths:
    with open(p, "r", encoding="utf-8") as f:
        for line in f:
            article = json.loads(line)
            docs.append(article["text"])
        if len(docs) >= 5000:
            break

print(f"Total documents loaded: {len(docs)}")


Total documents loaded: 5061


In [ ]:
# 1. Install Google Generative AI library
!pip install -q -U google-generativeai

In [ ]:
import time
import json
import os
import google.generativeai as genai
from tqdm import tqdm
from google.api_core.exceptions import ResourceExhausted, DeadlineExceeded

# ------------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------------
MODEL_NAME = "gemma-3-27b-it"

API_KEYS = [
    ""

    # "AIzaSy...PASTE_KEY_3_HERE",
]

SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Arabic_Data/arabic_summaries_gemma.jsonl"

genai.configure(api_key=API_KEYS[0])
model = genai.GenerativeModel(MODEL_NAME)

# ------------------------------------------------------------------
# DEBUGGED FUNCTION
# ------------------------------------------------------------------
def summarize_with_gemma(text):
    # Strict truncation for speed
    if len(text) > 4500:
        text = text[:4500]

    prompt = "لخص النص التالي باللغة العربية باختصار وشمولية:\n\n" + text

    for attempt in range(3):
        try:
            start_time = time.time()

            # DEBUG PRINT: Tell us we are trying
            # print(f"   ... sending request (attempt {attempt+1}) ...")

            # TIMEOUT ADDED: If no answer in 15 seconds, crash and retry
            # This prevents the "Stuck at 0%" issue
            response = model.generate_content(prompt, request_options={'timeout': 15})

            end_time = time.time()
            duration = end_time - start_time

            # Speed control (5 seconds per doc)
            required_wait = 5.0 - duration
            if required_wait > 0:
                time.sleep(required_wait)

            return response.text

        except ResourceExhausted:
            print(f"⚠️ Limit Hit. Waiting 20s...")
            time.sleep(20)
        except DeadlineExceeded:
            print(f"⚠️ Google timed out (took too long). Retrying...")
            time.sleep(5)
        except Exception as e:
            error_str = str(e)
            if "500" in error_str or "503" in error_str:
                time.sleep(5)
            else:
                print(f"❌ Error: {error_str}")
                return None
    return None

print(f"🚀 Starting Debugged Summarization...")

# Load progress
processed_count = 0
if os.path.exists(SAVE_PATH):
    with open(SAVE_PATH, "r", encoding="utf-8") as f:
        processed_count = sum(1 for _ in f)
    print(f"Resuming from document {processed_count}")

# Append mode
with open(SAVE_PATH, "a", encoding="utf-8") as f:
    for i, doc_text in enumerate(tqdm(docs[processed_count:], desc="Processing")):

        summary = summarize_with_gemma(doc_text)

        if summary:
            record = {"original_text": doc_text, "summary": summary}
            f.write(json.dumps(record, ensure_ascii=False) + "\n")
            f.flush()

🚀 Starting Debugged Summarization...


Processing:  17%|█▋        | 843/5061 [1:25:56<6:59:22,  5.97s/it]

❌ Error: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


Processing:  31%|███▏      | 1594/5061 [2:43:28<5:30:53,  5.73s/it]

❌ Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


...

Processing:  44%|████▎     | 2211/5061 [3:47:14<4:23:56,  5.56s/it]

❌ Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Processing:  79%|███████▉  | 4003/5061 [7:02:55<1:44:07,  5.91s/it]

❌ Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Processing: 100%|██████████| 5061/5061 [8:49:43<00:00,  6.28s/it]


In [ ]:
genai.configure(api_key="")

print("🔍 Checking available models for this key...")
try:
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"✅ Found: {m.name}")
except Exception as e:
    print(f"❌ Error: {e}")

🔍 Checking available models for this key...
✅ Found: models/gemini-2.5-flash
✅ Found: models/gemini-2.5-pro
✅ Found: models/gemini-2.0-flash-exp
✅ Found: models/gemini-2.0-flash
✅ Found: models/gemini-2.0-flash-001
✅ Found: models/gemini-2.0-flash-exp-image-generation
✅ Found: models/gemini-2.0-flash-lite-001
✅ Found: models/gemini-2.0-flash-lite
✅ Found: models/gemini-2.0-flash-lite-preview-02-05
✅ Found: models/gemini-2.0-flash-lite-preview
✅ Found: models/gemini-exp-1206
✅ Found: models/gemini-2.5-flash-preview-tts
✅ Found: models/gemini-2.5-pro-preview-tts
✅ Found: models/gemma-3-1b-it
✅ Found: models/gemma-3-4b-it
✅ Found: models/gemma-3-12b-it
✅ Found: models/gemma-3-27b-it
✅ Found: models/gemma-3n-e4b-it
✅ Found: models/gemma-3n-e2b-it
✅ Found: models/gemini-flash-latest
✅ Found: models/gemini-flash-lite-latest
✅ Found: models/gemini-pro-latest
✅ Found: models/gemini-2.5-flash-lite
✅ Found: models/gemini-2.5-flash-image-preview
✅ Found: models/gemini-2.5-flash-image
✅ Found: mod

In [1]:
%%capture
# Installs Unsloth, Xformers (for speed), and other necessary libraries
!pip install unsloth
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [1]:
#drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import json

# ------------------------------------------------------------------
# 1. CONFIGURATION (UPDATED FOR LONG CONTEXT)
# ------------------------------------------------------------------
# CHANGE 1: Increase context to 8192 (Approx 25k chars)
# Qwen 2.5 3B is small enough to handle this on Colab T4!
max_seq_length = 8192
dtype = None
load_in_4bit = True

# ------------------------------------------------------------------
# 2. LOAD MODEL
# ------------------------------------------------------------------
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# ------------------------------------------------------------------
# 3. PREPARE DATA
# ------------------------------------------------------------------
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Arabic_Data/arabic_summaries_gemma.jsonl"

data = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        try: data.append(json.loads(line))
        except: continue

print(f"Loaded {len(data)} documents.")
dataset = Dataset.from_list(data)

alpaca_prompt = """<|im_start|>system
لخص النص التالي باللغة العربية باختصار وشمولية.<|im_end|>
<|im_start|>user
{input}<|im_end|>
<|im_start|>assistant
{output}<|im_end|>"""

def formatting_prompts_func(examples):
    inputs = examples["original_text"]
    outputs = examples["summary"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = alpaca_prompt.format(input=input, output=output)
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched = True)





==((====))==  Unsloth 2025.12.7: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Loaded 5056 documents.


Map:   0%|          | 0/5056 [00:00<?, ? examples/s]

In [5]:
# ------------------------------------------------------------------
# 4. TRAIN (UPDATED SETTINGS)
# ------------------------------------------------------------------
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        # CHANGE 2: Reduce batch size to 1 because texts are huge now
        per_device_train_batch_size = 1,
        # CHANGE 3: Increase accumulation to keep training stable
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

print("🚀 Starting Training (Long Context Mode)...")
trainer.train()
print("✅ Training Finished!")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/5056 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 Starting Training (Long Context Mode)...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,056 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


...

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.295200
2,2.334500
3,2.414500
4,2.208700
5,2.568700
6,2.291100
7,2.143200
8,2.246700
9,2.206900
10,2.238000


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇█████
train/grad_norm,█▇▅▅▃▃▃▃▃▂▁▁▁▂▁▁▂▁▁▁▁▁▂▁▂▃▂▂▁▂▂▂▂▂▃▂▂▂▂▃
train/learning_rate,▂▄▇███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
train/loss,▇▇█▆▇▆▄▅▆▅▃▅▃▄▃▄▅▄▄▅▄▁▄▁▃▄▂▆▃▄▄▄▄▄▄▁▄▄▃▄
total_flos,6610352997801984.0
train/epoch,0.09494
train/global_step,60
train/grad_norm,0.27779
train/learning_rate,0.0
train/loss,2.0643


✅ Training Finished!


In [12]:
# 1. Enable Inference Mode
FastLanguageModel.for_inference(model)

# 2. Get the text
long_text = data[0]["original_text"]

# 3. TRUNCATION (CRITICAL FIX)
# We cut it to 20,000 chars to ensure the PROMPT doesn't get cut off.
if len(long_text) > 20000:
    long_text = long_text[:20000]

print(f"Input Length: {len(long_text)} chars (Safe for 8192 context)")

# Updated System Prompt to forbid foreign languages
prompt = f"""<|im_start|>system
لخص النص التالي باللغة العربية الفصحى حصراً. تجنب استخدام الكلمات الإنجليزية أو الصينية تماماً.<|im_end|>
<|im_start|>user
{long_text}<|im_end|>
<|im_start|>assistant
"""


inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

# 5. Generate with Anti-Loop Settings
print("⏳ Generating Verification Summary...")
outputs = model.generate(
    **inputs,
    max_new_tokens = 512,
    use_cache = True,
    temperature = 0.5,       # Low temperature = Focused
    repetition_penalty = 1.2 # > 1.0 prevents loops
)

result = tokenizer.batch_decode(outputs)[0]
summary = result.split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", "")

print("\n--- NEW SUMMARY ---")
print(summary)

Input Length: 20000 chars (Safe for 8192 context)
⏳ Generating Verification Summary...

--- NEW SUMMARY ---
الماء هو مادة أساسية وخطيرة لحياة الأرض، تشكل 71٪ من سطحها، وتقصد به المحيطات والبحار والبحيرات والأنهار والم groundwater. تمتد حياته على كامل الأرض، لكنها الأكثر شيوعًا في المحيطات، حيث تشكل 96.5٪ من مياه الأرض. يلعب الماء أيضًا دورًا حيويًا في الحفاظ على الحياة على الأرض، حيث يساعد في توفير الطاقة اللازمة لعملية التمثيل الضوئي والحصول على الطاقة من خلال عملية الاحتراق الأولية، مما يجعله أساسًا للإ饱和 الكيميائي والإنزيمات.
تقوم مجموعة متنوعة من الكائنات الحية بتناول الماء، خاصة النباتات والحيوانات المائية. يعتمد الماء أيضًا على موقعه الجغرافي في تكوين habitats ونظم بيئية، مثل الغابات المائية والبحرية. تختلف تركيزات الملح في الماء حسب الموقع الجغرافي، حيث يحتوي الماء المالح بكفاءة عالية على الملوثات، بينما يحتوي الماء العذب على أقل كمية. يلعب الماء أيضًا دورًا حيويًا في الحفاظ على الحضارة البشرية، حيث يعتبر المصدر الأساسي لمياه الشرب، واستخدامه في الزراعة والصناعات المختلفة.
منذ ب

In [11]:
data[0]["summary"]

'الماء هو مركب كيميائي أساسي للحياة على الأرض، يتكون من ذرتي هيدروجين وذرة أكسجين (H2O). يغطي 71% من سطح الأرض، ويتواجد في ثلاث حالات: سائل (أكثر شيوعًا)، صلب (جليد)، وغازي (بخار ماء). يلعب الماء دورًا حيويًا في تنظيم المناخ وتوزيع الحرارة عبر التيارات المائية.\n\nلا يقتصر وجود الماء على الأرض، بل يتواجد بكميات هائلة في الكون، وينشأ غالبًا في النجوم. تم اكتشاف كميات ضخمة من بخار الماء في الفضاء، وحتى أشكال افتراضية للماء تحت ضغوط وحرارة عالية.\n\nوجود الماء بحالاته الثلاث على الأرض، بفضل موقعها في النطاق الصالح للحياة، هو شرط أساسي لظهور الحياة واستمرارها. كما توجد دلائل على وجود الماء السائل تحت سطح بعض الأقمار مثل إنسيلادوس وتيتان وأوروبا.\n\nيشكل نقص المياه العذبة تحديًا عالميًا، حيث يفتقر الملايين إلى الوصول إلى مياه شرب آمنة.\n'

In [25]:
save_path = "/content/drive/MyDrive/Colab Notebooks/Arabic_Data/Qwen_Arabic_Summary_LoRA"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to {save_path}")

Model saved to /content/drive/MyDrive/Colab Notebooks/Arabic_Data/Qwen_Arabic_Summary_LoRA


In [2]:
#drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# =========================================================
# Full evaluation on arabic_summaries_gemma_2000 (sample 100)
# Metrics: ROUGE + BERTScore + LLM-as-a-Judge
# =========================================================

# -------- paths --------
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/Arabic_Data/Qwen_Arabic_Summary_LoRA"
DATA_PATH  = "/content/drive/MyDrive/Colab Notebooks/Arabic_Data/arabic_summaries_gemma_2000.jsonl"

# -------- eval settings --------
N = 100
BATCH_SIZE = 2
MAX_NEW_TOKENS = 128
SRC_CHAR_LIMIT = 2500
SEED = 42

# -------- install deps --------
!pip -q install transformers accelerate evaluate rouge_score bert_score google-generativeai google-api-core

import os, json, random, re, time
import numpy as np
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import evaluate

import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted, DeadlineExceeded

# -------- 1) Load dataset --------
records = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            records.append(json.loads(line))

random.Random(SEED).shuffle(records)
sample = records[:min(N, len(records))]

sources = [r["original_text"][:SRC_CHAR_LIMIT] for r in sample]
references = [r["summary"] for r in sample]

print(f"Loaded {len(records)} total | Evaluating {len(sample)} samples")

# -------- 2) Load saved model (NO training) --------
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)
model.eval()

def build_prompt(src: str) -> str:
    return f"""<|im_start|>system
لخص النص التالي بالعربية الفصحى فقط.
ممنوع إضافة أي معلومات غير موجودة في النص.
<|im_end|>
<|im_start|>user
{src}
<|im_end|>
<|im_start|>assistant
"""

@torch.inference_mode()
def generate_summaries(texts):
    preds = []
    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Generating"):
        batch = texts[i:i+BATCH_SIZE]
        prompts = [build_prompt(t) for t in batch]
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=4096,
        ).to(model.device)

        out = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            temperature=0.0,
            use_cache=True,
        )

        decoded = tokenizer.batch_decode(out, skip_special_tokens=False)
        for d in decoded:
            pred = d.split("<|im_start|>assistant")[-1]
            pred = pred.split("<|im_end|>")[0].strip()
            preds.append(pred)
    return preds

predictions = generate_summaries(sources)

# -------- 3) ROUGE --------
rouge = evaluate.load("rouge")
rouge_scores = rouge.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True
)

print("\nROUGE:")
for k, v in rouge_scores.items():
    print(f"  {k}: {v:.4f}")

# -------- 4) BERTScore (Arabic) --------
bertscore = evaluate.load("bertscore")
bs = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="ar",
    model_type="bert-base-multilingual-cased",
)

print("\nBERTScore (mean):")
print(f"  Precision: {float(np.mean(bs['precision'])):.4f}")
print(f"  Recall:    {float(np.mean(bs['recall'])):.4f}")
print(f"  F1:        {float(np.mean(bs['f1'])):.4f}")

# -------- 5) LLM-as-a-Judge (Gemma) --------
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "").strip()
if not GEMINI_API_KEY:
    print("\nLLM-as-a-Judge: skipped (no API key)")
else:
    genai.configure(api_key=GEMINI_API_KEY)
    judge_model = genai.GenerativeModel("gemma-3-27b-it")

    def judge_one(src, pred, retries=2):
        src = src[:1800]
        pred = pred[:700]
        prompt = f"""أنت مُحكِّم جودة تلخيص.
أرجع JSON فقط:
{{"score": <int 1..10>}}

قيّم الملخص مقابل النص الأصلي.
ممنوع اختلاق معلومات غير موجودة.

النص الأصلي:
{src}

الملخص:
{pred}
"""
        for _ in range(retries):
            try:
                resp = judge_model.generate_content(prompt)
                text = (resp.text or "").strip()
                m = re.search(r"\{[\s\S]*\}", text)
                if not m:
                    return None
                score = json.loads(m.group(0)).get("score", None)
                if isinstance(score, (int, float)) and 1 <= int(score) <= 10:
                    return int(score)
            except (ResourceExhausted, DeadlineExceeded):
                time.sleep(2)
            except Exception:
                time.sleep(1)
        return None

    scores = []
    for s, p in tqdm(list(zip(sources, predictions)), desc="Judging"):
        sc = judge_one(s, p)
        if sc is not None:
            scores.append(sc)

    if scores:
        print("\nLLM-as-a-Judge (Gemma):")
        print(f"  Scored {len(scores)}/{len(predictions)}")
        print(f"  Average score: {float(np.mean(scores)):.2f} / 10")
    else:
        print("\nLLM-as-a-Judge: No scores parsed")


Loaded 1998 total | Evaluating 100 samples


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Generating: 100%|██████████| 50/50 [29:12<00:00, 35.05s/it]


...

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


BERTScore (mean):
  Precision: 0.7791
  Recall:    0.7226
  F1:        0.7497

LLM-as-a-Judge: skipped (no API key)


In [1]:
!pip install -q bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.0 MB/s eta 0:00:00


In [4]:
import os
os.environ["GEMINI_API_KEY"] = ""


In [5]:
!pip -q install google-generativeai google-api-core

import os, re, json, time
import numpy as np
from tqdm import tqdm
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted, DeadlineExceeded

# --- check key ---
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "").strip()
if not GEMINI_API_KEY:
    raise RuntimeError("GEMINI_API_KEY is not set")

# --- init model ---
genai.configure(api_key=GEMINI_API_KEY)
judge_model = genai.GenerativeModel("gemma-3-27b-it")

def judge_one(src, pred, retries=2):
    src = src[:1800]
    pred = pred[:700]

    prompt = f"""أنت مُحكِّم جودة تلخيص.
أرجع JSON فقط بالشكل:
{{"score": <int 1..10>}}

قيّم الملخص مقابل النص الأصلي.
ممنوع اختلاق معلومات غير موجودة.

النص الأصلي:
{src}

الملخص:
{pred}
"""
    for _ in range(retries):
        try:
            resp = judge_model.generate_content(prompt)
            text = (resp.text or "").strip()
            m = re.search(r"\{[\s\S]*\}", text)
            if not m:
                return None
            score = json.loads(m.group(0)).get("score", None)
            if isinstance(score, (int, float)) and 1 <= int(score) <= 10:
                return int(score)
        except (ResourceExhausted, DeadlineExceeded):
            time.sleep(2)
        except Exception:
            time.sleep(1)
    return None

# --- run judge (limit N if you want) ---
N = min(30, len(predictions))  # change or remove
scores = []

for src, pred in tqdm(list(zip(sources[:N], predictions[:N])), desc="Judging"):
    s = judge_one(src, pred)
    if s is not None:
        scores.append(s)

if scores:
    print(f"LLM-as-a-Judge average score: {float(np.mean(scores)):.2f} / 10")
else:
    print("LLM-as-a-Judge: No scores parsed")


Judging: 100%|██████████| 30/30 [00:50<00:00,  1.68s/it]

LLM-as-a-Judge average score: 8.44 / 10
